In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv("Sensex15.csv").tail(1000)
df.columns

Index(['Day_Open', 'Day_High', 'Day_Low', 'Day_Close', 'Day_SuperTrend',
       'Day_Candle', 'Day_CandleBody', 'Day_5EMA', 'Day_15EMA', 'Day_20EMA',
       'Day_50EMA', 'Day_200EMA', 'Day_RSI', 'Day_upper_shadow_pr',
       'Day_lower_shadow_pr', 'Day_Candle_Signal', 'Day_Prev_Candle_Signal',
       'Day_Prev_Candle', 'Day_Date', 'Day_SwingMin', 'Day_SwingMax',
       'Datetime', 'Open', 'High', 'Low', 'Close', 'First_Candle_High',
       'First_Candle_Low', 'Pivot', 'Candle', 'CandleBody', '5EMA', '15EMA',
       '20EMA', '50EMA', '200EMA', 'RSI', 'SuperTrend', 'upper_shadow_pr',
       'lower_shadow_pr', 'Candle_Signal', 'Small_Swing_Min',
       'Small_Swing_Max'],
      dtype='object')

In [23]:
df.CandleBody.max()

695.4799999999959

In [24]:
df.CandleBody.min()

33.69999999999709

In [25]:
df.CandleBody.mean()

123.55389999999994